In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import sys
sys.path.append('/Users/david/miniconda3/lib/python3.8/site-packages')

import cv2 as cv

from ipywidgets import interact
import ipywidgets as widgets

# Idée

Trouver au début d'un ensemble d'opérations de traitement, les bonnes valeurs de seuil sur lesquelles moyenner. En effet, la luminosité et les conditions peuvent changer d'une série de photos à l'autre

$\rightarrow$ Définir un score dépendant des paramètres pertinents pour définir une plage sur laquelle moyenner (plage sur laquelle ils sont signifiants)

## Données à prendre en compte

- Nombre d'ellipses pertinentes détectées (doit être $2$)
- Nombre d'ellipses totales détectées (s'il est trop élevé : bruit qui peut altérer la mesure)
- Stabilité des angles des ellipses (variance sur la fenêtre)
- Stabilité des mesures d'excentricité et de concentricité (variance sur la fenêtre)
- Redondant mais éventuellement plus pertinent que le point précédent : précision de l'approximation linéaire des différents axes

## Scores

But : vérifier (1) qu'on mesure bien les bonnes données et (2) qu'on continue de mesurer les mêmes éléments (variations régulières) au fil de l'évolution du seuil.

$\rightarrow$ Un score __pertinence du domaine__ (dépend des deux premiers points)

$\rightarrow$ Un score __cohérence des mesures__ (dépend de 10 valeurs : les 2 axes, les 2 coordonnées du centre et les angles [quel modulo ?] de chaque ellipse)

$\rightarrow$ On moyenne l'excentricité et l'écart de concentricité sur le domaine déterminé.

## Plan de calcul des scores

- Nombre d'ellipses pertinentes : doit être 2 (si plus que 2, filtre à ajuster)
- Nombre total : __tendance exponentielle__. Définir une valeur max (interprétation : au-delà d'une certaine valeur, le bruit aussi détecté devient susceptible d'influencer significativement les mesures réalisées).
- Pour la longueur des $4$ axes : on s'attend à une __tendance affine__ sur la partie d'intérêt et à des variations plus rapides en dehors. Il s'agit de déterminer la frontière entre ces tendances. Il s'agit de déterminer la meilleure régression affine (calculée sur le domaine d'intérêt) et voir à quel moment, les données commencent à s'en éloigner.
- Pour les coordonnées des centres et les angles, on s'attend à une __constante__ sur la partie d'intérêt. On va pouvoir travailler sur la _variance des données_.

# Autres éléments de preprocessing

- Filtrer le bruit ?
- Recadrage ?

## Filtrage de l'image

# Dossier définitions

1. Récupération de l'image
2. Preprocessing : conversion en niveaux de gris, seuillage
3. Détermination des contours
4. Détection des ellipses contenues par la figure en fonction des contours
5. Filtrage des ellipses en fonction de ses caractéristiques géométriques (taille des axes et excentricité pas trop élevée) puis tracé des différentes ellipses pertinentes
6. Exctraction des caractéristiques géométriques des ellipses pertinentes

Questions à régler :

- Quel seuil ? Ou plutôt : quelle fenêtre de seuils sur laquelle moyenner les résultats ?
- Recadrage, élimination du bruit en amont ?

Détails des choix techniques :

- VS utilisait Canny : semble être une mauvaise idée.
- Détection de cercles (HoughCircles), rectangles (BoundRectangles) ou rectangles avec rotation (MinAreaRectangles) donnent des moins bons résultats, respectivement :
    - mausvaise détection (multitudes de cercles non pertinents trouvés et difficiles à filtrer) ;
    - ne donne pas les bonnes caractéristiques (au plus l'écart de concentricité) ;
    - performances inférieures aux ellipses (l'ellipses permet de moyenner sur le contour).
- Rétroéclairage fort sans éclairage supérieur :
    - évite les ombres (cercles intérieur et extérieur et ombres des objets environnants) ;
    - évite de voir les défauts de la pièces qui compliquent la mesure ;
    - homogénéité du fond.